<a href="https://colab.research.google.com/github/tadamjanic/Syllabus/blob/master/0_Modificacion_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import time
from pandas import DataFrame
from datetime import datetime, timedelta
from google.colab import drive
from google.colab import files
import networkx as nx
# se registra el cambio
pd.set_option('display.float_format', lambda x: '%.1f' % x)
pd.set_option('display.max_columns', 500)

In [ ]:
# MONTAR DIRECTORIO
from google.colab import drive
drive.mount('/gdrive/')

%cd /gdrive/MyDrive/Operaciones/Assessment/Los Bronces/

%ls # Mostramos los archivos

Mounted at /gdrive/
/gdrive/.shortcut-targets-by-id/1IPV1ca7izG3Vx1EcLfQjCfgqtuPKWiRK/Operaciones/Assessment/Los Bronces
'3. Planificación y Control'/   '7. Requisitos TI'/
'4. Información Empresa'/        DestinosComunes.xlsx
'5. Reuniones y Visitas'/        Dums_year_month.xlsx
'6. Investigación y Análisis'/   prop_chancadores.xlsx


# Importante: Apretar ctrl + shift + click en las flechas, para minimizar y luego expandir todo, buscar con ctrl + f hardcodeado y al ver los errores ir reemplazando con lo que aparece en la celda anterior

In [ ]:
path_db = '4. Información Empresa/BD 2021-2022/operativas/data cruda/'
path_db2 = '4. Información Empresa/BD 2021-2022/instantaneas/'

rel_path = './6. Investigación y Análisis/Insumos Simulador_3/Resultados/Otros/'

### Acceso bases

In [ ]:
df_state = pd.read_csv(f'{path_db}Shiftshiftstate.csv', sep=';')
df_state = df_state[['FieldId','ShiftId','FieldEqmt','FieldAuxeqmt','FieldReason','FieldStatus']]

#### En eqmt está la información de palas ###
df_eqmt = pd.read_csv(f'{path_db}Shiftshifteqmt.csv',sep=';')
df_eqmt = df_eqmt[['Id','FieldId','FieldEqmttype']].drop_duplicates()
df_eqmt = df_eqmt.rename(columns={'FieldId':'FieldIdEqmt'})

df_aux = pd.read_csv(f'{path_db}Shiftshiftaux.csv', sep=';', encoding='latin-1', error_bad_lines=False)    #info chancado
df_aux = df_aux[(df_aux['FieldId']=='CHANC-2')|(df_aux['FieldId']=='CHANCADOR')]
df_aux = df_aux[['Id','FieldId','FieldUnit']]
df_aux = df_aux.rename(columns={'FieldId':'FieldIdEqmt','FieldUnit':'FieldEqmttype'})
df_aux['Id'] = df_aux['Id'].astype(int)

df_loc = pd.read_csv(f'{path_db}Shiftshiftloc.csv', sep = ';')
df_loc = df_loc[['Id','FieldId','FieldRegion','FieldUnit']]

df_loc_2 = pd.read_csv(f'{path_db}Shiftshiftloc.csv', sep = ';')


df_enum = pd.read_csv(f'{path_db2}Enum.csv',sep=';')
df_enum = df_enum[['Id','Description']].drop_duplicates()

#df_pit_loc = pd.read_csv(f'{path_db3}PITPitloc.csv', sep=';')


#df_enum_2 = pd.read_excel(f'{path_db2}TablasFijas2107.xlsx')

## En aux está la info de chancadores
df_aux_2 = pd.read_csv(f'{path_db}Shiftshiftaux.csv', sep=';', encoding='latin-1', error_bad_lines=False)    #info chancado
df_aux_2 = df_aux_2[(df_aux_2['FieldId']=='CHANC-2')|(df_aux_2['FieldId']=='CHANCADOR')]
df_aux_2 = df_aux_2[['Id','FieldId','FieldUnit']]
df_aux_2 = df_aux_2.rename(columns={'FieldId':'FieldIdEqmt','FieldUnit':'FieldEqmttype'})

df_loads = pd.read_csv(f'{path_db}Shiftshiftload.csv', sep=';') 
df_loads['Tonnage'] = df_loads['FieldLsizetons'].astype(str).str.replace(",",".").astype(float)

df_dumps = pd.read_csv(f'{path_db}Shiftshiftdump.csv', sep=';')
df_dumps['Tonnage'] = df_dumps['FieldLsizetons'].astype(str).str.replace(",",".").astype(float)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (9,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (11,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


###Cruces necesarios

#### df_dumps

In [ ]:
df_dumps.head(5)

,ShiftId,Id,DbKey,FieldId,FieldTruck,FieldLoc,FieldGrade,FieldLoadrec,FieldExcav,FieldBlast,FieldBay,FieldTons,FieldTimearrive,FieldTimedump,FieldTimeempty,FieldCalctravtime,FieldLoad,FieldExtraload,FieldDist,FieldEfh,FieldLoadtype,FieldToper,FieldEoper,FieldOrigasn,FieldReasnby,FieldLocactlc,FieldLocacttp,FieldLocactrl,FieldAudit,FieldLsizetons,FieldDlock,FieldElock,FieldEdlock,FieldRlock,FieldTimearrivemobile,FieldTimedumpmobile,FieldTimeemptymobile,FieldPathtravtime,FieldExptraveldist,FieldExptraveltime,Tonnage
0,210101002,2101010020001340352,1340352,NaN,2101010020000013616,2101010020000007908,2101010020000055856,2101010020001864544,2101010020000033264,2101010020000009052,-1,299,1798,1854,1893,246,90,0,1026,1036,412,2101010020000013868,2101010020000259400,NaN,NaN,NaN,0,NaN,-1,"298,8543",NINGUNA,PAB05,PRIM,NINGUNA,1609514997,1609515052,1609515091,175,1026,141,298.9
1,210101002,2101010020001517808,1517808,NaN,2101010020000025752,2101010020000004300,2101010020000058448,2101010020002077232,2101010020000036124,2101010020000004740,-1,309,2020,2020,2056,629,93,0,3409,3266,412,2101010020000253632,2101010020000967740,NaN,NaN,NaN,0,NaN,-1,"298,6113",NINGUNA,NINGUNA,NINGUNA,NINGUNA,1609533213,-1,1609533254,726,3343,713,298.6
2,210101002,2101010020001863560,1863560,NaN,2101010020000029200,2101010020000007908,2101010020000055856,2101010020002049884,2101010020000033264,2101010020000009052,-1,292,2011,2090,2126,246,90,0,1026,1036,412,2101010020000016772,2101010020000259400,NaN,NaN,NaN,0,NaN,-1,"298,8543",NINGUNA,PAB05,PRIM,NINGUNA,1609533209,1609533288,1609533325,182,1026,139,298.9
3,210101002,2101010020001869236,1869236,NaN,2101010020000027744,2101010020000007908,2101010020000057284,2101010020002432868,2101010020000033768,2101010020000006588,-1,297,2070,2150,2207,1461,90,0,5771,6061,412,2101010020000027996,2101010020000244252,NaN,NaN,NaN,0,NaN,-1,"298,1546",NINGUNA,NINGUNA,PRIM DOS,NINGUNA,1609533269,1609533350,1609533406,1075,246,38,298.2
4,210101002,2101010020001871384,1871384,NaN,2101010020000021828,2101010020000003508,2101010020000264792,2101010020002435124,2101010020000034132,2101010020000006236,-1,0,2179,2217,2288,1594,92,0,6757,6892,412,2101010020000022080,2101010020000256016,NaN,NaN,NaN,0,NaN,2101010020032728912,"303,0774",NINGUNA,NINGUNA,PRIM,NINGUNA,1609533378,1609533416,1609533486,1185,6757,1064,303.1


##### Agregar Truck y Excav

In [ ]:
df_dumps = df_dumps.drop(columns=['FieldId'])
print('Filas antes de cruzar con eqmt:',len(df_dumps))
df_dumps = df_dumps.merge(df_eqmt,left_on='FieldTruck', right_on='Id', how='inner')
print('Filas despues de cruzar con eqmt:',len(df_dumps), 'No se deberia perder ninguna')
df_dumps = df_dumps.drop(columns=['Id_y'])
df_dumps = df_dumps.rename(columns={'Id_x':'Id'})
df_dumps = df_dumps.merge(df_enum, left_on='FieldEqmttype', right_on='Id', how='inner')
print('Filas despues de cruzar con enum:',len(df_dumps), 'No se deberia perder ninguna')
df_dumps = df_dumps.drop(columns=['Id_y'])
df_dumps = df_dumps.rename(columns={'Id_x':'Id'})
df_dumps = df_dumps.rename(columns={'FieldEqmttype': 'FieldEqmtTypeTruck','Description':'EqmtTypeTruck', 'FieldIdEqmt':'Truck'})

Filas antes de cruzar con eqmt: 537091
Filas despues de cruzar con eqmt: 537091 No se deberia perder ninguna
Filas despues de cruzar con enum: 537091 No se deberia perder ninguna


In [ ]:
print('Filas antes de cruzar con eqmt:',len(df_dumps))
df_dumps = df_dumps.merge(df_eqmt,left_on='FieldExcav', right_on='Id', how='inner')
print('Filas despues de cruzar con eqmt:',len(df_dumps), 'No se deberia perder ninguna')
df_dumps = df_dumps.drop(columns=['Id_y'])
df_dumps = df_dumps.rename(columns={'Id_x':'Id'})
df_dumps = df_dumps.merge(df_enum, left_on='FieldEqmttype', right_on='Id', how='inner')
print('Filas despues de cruzar con enum:',len(df_dumps), 'No se deberia perder ninguna')
df_dumps = df_dumps.rename(columns={'FieldEqmttype': 'FieldEqmtTypeExcav','Description':'EqmtTypeExcav', 'FieldIdEqmt':'Excav'})
df_dumps = df_dumps.drop(columns=['Id_y'])
df_dumps = df_dumps.rename(columns={'Id_x':'Id'})

Filas antes de cruzar con eqmt: 537091
Filas despues de cruzar con eqmt: 537091 No se deberia perder ninguna
Filas despues de cruzar con enum: 537091 No se deberia perder ninguna


##### Agregar información Dump(falta region)

In [ ]:
print('Filas antes de cruzar con shift_loc (dump):',len(df_dumps))
df_dumps = df_dumps.merge(df_loc,  left_on='FieldLoc', right_on='Id',how='inner')
df_dumps = df_dumps.drop(columns=['Id_y'])
df_dumps = df_dumps.rename(columns={'Id_x':'Id'})
print('Filas despues de cruzar con shift_loc:',len(df_dumps), 'No se deberia perder ninguna')
df_dumps = df_dumps.merge(df_enum, left_on='FieldUnit', right_on='Id', how='inner')
print('Filas despues de cruzar con enum:',len(df_dumps), 'No se deberia perder ninguna')
df_dumps = df_dumps.rename(columns={'Description':'DumpLocationUnit','FieldId':'DumpLocation','FieldRegion':'FieldRegionDump','FieldUnit':'FieldUnitDump'})
df_dumps = df_dumps.drop(columns=['Id_y'])
df_dumps = df_dumps.rename(columns={'Id_x':'Id'})

Filas antes de cruzar con shift_loc (dump): 537091
Filas despues de cruzar con shift_loc: 537091 No se deberia perder ninguna
Filas despues de cruzar con enum: 537091 No se deberia perder ninguna


In [ ]:
df_dumps.head(5)

,ShiftId,Id,DbKey,FieldTruck,FieldLoc,FieldGrade,FieldLoadrec,FieldExcav,FieldBlast,FieldBay,FieldTons,FieldTimearrive,FieldTimedump,FieldTimeempty,FieldCalctravtime,FieldLoad,FieldExtraload,FieldDist,FieldEfh,FieldLoadtype,FieldToper,FieldEoper,FieldOrigasn,FieldReasnby,FieldLocactlc,FieldLocacttp,FieldLocactrl,FieldAudit,FieldLsizetons,FieldDlock,FieldElock,FieldEdlock,FieldRlock,FieldTimearrivemobile,FieldTimedumpmobile,FieldTimeemptymobile,FieldPathtravtime,FieldExptraveldist,FieldExptraveltime,Tonnage,Truck,FieldEqmtTypeTruck,EqmtTypeTruck,Excav,FieldEqmtTypeExcav,EqmtTypeExcav,DumpLocation,FieldRegionDump,FieldUnitDump,DumpLocationUnit
0,210101002,2101010020001340352,1340352,2101010020000013616,2101010020000007908,2101010020000055856,2101010020001864544,2101010020000033264,2101010020000009052,-1,299,1798,1854,1893,246,90,0,1026,1036,412,2101010020000013868,2101010020000259400,NaN,NaN,NaN,0,NaN,-1,"298,8543",NINGUNA,PAB05,PRIM,NINGUNA,1609514997,1609515052,1609515091,175,1026,141,298.9,CDH41,135,KOM 930E,PAB05,154,BUCYRUS 495B/HR,PRIM,2101010020000003068,267,Chancador
1,210101002,2101010020002075348,2075348,2101010020000013616,2101010020000007908,2101010020000055856,2101010020002428536,2101010020000033264,2101010020000009052,-1,297,2278,3054,3093,246,90,0,1026,1036,412,2101010020000013868,2101010020000259400,NaN,NaN,NaN,0,NaN,-1,"298,8543",NINGUNA,PAB05,PRIM,NINGUNA,1609515676,1609516253,1609516292,198,1026,146,298.9,CDH41,135,KOM 930E,PAB05,154,BUCYRUS 495B/HR,PRIM,2101010020000003068,267,Chancador
2,210101002,2101010020002437984,2437984,2101010020000013616,2101010020000007908,2101010020000055856,2101010020002968748,2101010020000033264,2101010020000009052,-1,290,3497,3529,3565,246,90,0,1026,1036,412,2101010020000013868,2101010020000259400,NaN,NaN,NaN,0,NaN,-1,"298,8543",NINGUNA,PAB05,PRIM,NINGUNA,1609516695,1609516728,1609516763,185,1026,160,298.9,CDH41,135,KOM 930E,PAB05,154,BUCYRUS 495B/HR,PRIM,2101010020000003068,267,Chancador
3,210101002,2101010020003156284,3156284,2101010020000013616,2101010020000007908,2101010020000055856,2101010020003516152,2101010020000033264,2101010020000009052,-1,300,4063,4347,4384,246,90,0,1026,1036,412,2101010020000013868,2101010020000259400,NaN,NaN,NaN,0,NaN,-1,"298,8543",NINGUNA,PAB05,PRIM,NINGUNA,1609517261,1609517545,1609517582,176,1026,144,298.9,CDH41,135,KOM 930E,PAB05,154,BUCYRUS 495B/HR,PRIM,2101010020000003068,267,Chancador
4,210101002,2101010020003526952,3526952,2101010020000013616,2101010020000007908,2101010020000055856,2101010020003889052,2101010020000033264,2101010020000009052,-1,290,4890,4918,4957,246,90,0,1026,1036,412,2101010020000013868,2101010020000259400,NaN,NaN,NaN,0,NaN,-1,"298,8543",NINGUNA,PAB05,PRIM,NINGUNA,1609518089,1609518117,1609518156,178,1026,143,298.9,CDH41,135,KOM 930E,PAB05,154,BUCYRUS 495B/HR,PRIM,2101010020000003068,267,Chancador


##### Agregar información blast (falta region)

In [ ]:
print('Filas antes de cruzar con shift_loc (blast):',len(df_dumps), 'No se deberia perder ninguna')
df_dumps = df_dumps.merge(df_loc,  left_on='FieldBlast', right_on='Id',how='inner')
df_dumps = df_dumps.drop(columns=['Id_y'])
df_dumps = df_dumps.rename(columns={'Id_x':'Id'})
print('Filas despues de cruzar con shift_loc:',len(df_dumps), 'No se deberia perder ninguna')
df_dumps = df_dumps.merge(df_enum, left_on='FieldUnit', right_on='Id', how='inner')
print('Filas despues de cruzar con enum:',len(df_dumps), 'No se deberia perder ninguna')
df_dumps = df_dumps.rename(columns={'Description':'BlastLocationUnit','FieldId':'BlastLocation','FieldRegion':'FieldRegionBlast','FieldUnit':'FieldUnitBlast'})
df_dumps = df_dumps.drop(columns=['Id_y'])
df_dumps = df_dumps.rename(columns={'Id_x':'Id'})

Filas antes de cruzar con shift_loc (blast): 537091 No se deberia perder ninguna
Filas despues de cruzar con shift_loc: 537091 No se deberia perder ninguna
Filas despues de cruzar con enum: 537091 No se deberia perder ninguna


##### Agregar region dump y blast

In [ ]:
# Para obtener la fase y region de dump y blast.
df_loc_2.head(2)

,ShiftId,Id,DbKey,FieldId,FieldPit,FieldRegion,FieldElev,FieldUnit,FieldStatus,FieldReason,FieldReasonrec
0,210101002,2101010020000002628,2628,DLB,-1,-1,3000,273,248,0,-1
1,210101002,2101010020000002716,2716,DONOSO 2,2101010020000002628,-1,0,274,248,0,-1


In [ ]:
df_dumps.head(1)

,ShiftId,Id,DbKey,FieldTruck,FieldLoc,FieldGrade,FieldLoadrec,FieldExcav,FieldBlast,FieldBay,FieldTons,FieldTimearrive,FieldTimedump,FieldTimeempty,FieldCalctravtime,FieldLoad,FieldExtraload,FieldDist,FieldEfh,FieldLoadtype,FieldToper,FieldEoper,FieldOrigasn,FieldReasnby,FieldLocactlc,FieldLocacttp,FieldLocactrl,FieldAudit,FieldLsizetons,FieldDlock,FieldElock,FieldEdlock,FieldRlock,FieldTimearrivemobile,FieldTimedumpmobile,FieldTimeemptymobile,FieldPathtravtime,FieldExptraveldist,FieldExptraveltime,Tonnage,Truck,FieldEqmtTypeTruck,EqmtTypeTruck,Excav,FieldEqmtTypeExcav,EqmtTypeExcav,DumpLocation,FieldRegionDump,FieldUnitDump,DumpLocationUnit,BlastLocation,FieldRegionBlast,FieldUnitBlast,BlastLocationUnit
0,210101002,2101010020001340352,1340352,2101010020000013616,2101010020000007908,2101010020000055856,2101010020001864544,2101010020000033264,2101010020000009052,-1,299,1798,1854,1893,246,90,0,1026,1036,412,2101010020000013868,2101010020000259400,NaN,NaN,NaN,0,NaN,-1,"298,8543",NINGUNA,PAB05,PRIM,NINGUNA,1609514997,1609515052,1609515091,175,1026,141,298.9,CDH41,135,KOM 930E,PAB05,154,BUCYRUS 495B/HR,PRIM,2101010020000003068,267,Chancador,STK_DONOSO_AL,2101010020000003068,268,Stock


In [ ]:
df = df_loc_2.values.tolist()
lista_fases = []
lista_fases_1 = []
lista_pit = []
lista_pit_1 = []
for fila in df:
  Fpit = fila[4]
  Ffase = fila[5]
  fieldid = fila[3]
  Id = fila[1]
  if Ffase == -1 and Fpit != -1 and Id not in lista_fases_1:
    lista_fases.append([Id,fieldid])
  if Ffase == -1 and Fpit == -1 and Id not in lista_pit_1:
    lista_pit.append([Id,fieldid])
 


In [ ]:
df_fases = pd.DataFrame(lista_fases,columns=['Id_1','fase'])
#df_rajo = pd.DataFrame(lista_pit,columns=['Id_1','PitBlast'])

In [ ]:
df_fases.head(5)

,Id_1,fase
0,2101010020000002716,DONOSO 2
1,2101010020000003068,MINA
2,2101010020000003684,INFIERNILLO 7A
3,2101010020000005180,INFIERNILLO 5
4,2101010020000005620,INFIERNILLO 6


In [ ]:
print('Filas antes de cruzar con fases de df_loc (blast):',len(df_dumps))
df_dumps = df_dumps.merge(df_fases,  left_on='FieldRegionBlast', right_on='Id_1',how='inner')
print('Filas despues de cruzar con df_loc:',len(df_dumps), 'Solo se pierden filas si una fase no esta en shift_loc, podría ser por tener menos rango de fecha en shift_loc')
df_dumps = df_dumps.drop(columns=['Id_1'])
df_dumps = df_dumps.rename(columns={'fase':'RegionBlast'})
print(len(df_dumps))

Filas antes de cruzar con fases de df_loc (blast): 537091
Filas despues de cruzar con df_loc: 498241 Solo se pierden filas si una fase no esta en shift_loc, podría ser por tener menos rango de fecha en shift_loc
498241


In [ ]:
print('Filas antes de cruzar con fases de df_loc (dump):',len(df_dumps))
df_dumps = df_dumps.merge(df_fases,  left_on='FieldRegionDump', right_on='Id_1',how='inner')
print('Filas despues de cruzar con df_loc:',len(df_dumps), 'Solo se pierden filas si una fase no esta en shift_loc, podría ser por tener menos rango de fecha en shift_loc')
df_dumps = df_dumps.drop(columns=['Id_1'])
df_dumps = df_dumps.rename(columns={'fase':'RegionDump'})
print(len(df_dumps))

Filas antes de cruzar con fases de df_loc (dump): 498241
Filas despues de cruzar con df_loc: 494313 Solo se pierden filas si una fase no esta en shift_loc, podría ser por tener menos rango de fecha en shift_loc
494313


##### Agregar tipo de material transportado

In [ ]:
##### para agregar tipo de material transportado
print('Filas antes de cruzar con enum:',len(df_dumps))
df_dumps = df_dumps.merge(df_enum,  left_on='FieldLoad', right_on='Id',how='inner')
print('Filas despues de cruzar con enum:',len(df_dumps), 'No deberia perder filas')
df_dumps = df_dumps.drop(columns=['Id_y'])
df_dumps = df_dumps.rename(columns={'Description':'MaterialType','Id_x':'Id'})

Filas antes de cruzar con enum: 494313
Filas despues de cruzar con enum: 494313 No deberia perder filas


##### Agregar Rajo blast y dump

In [ ]:
df_dumps.head(2)

,ShiftId,Id,DbKey,FieldTruck,FieldLoc,FieldGrade,FieldLoadrec,FieldExcav,FieldBlast,FieldBay,FieldTons,FieldTimearrive,FieldTimedump,FieldTimeempty,FieldCalctravtime,FieldLoad,FieldExtraload,FieldDist,FieldEfh,FieldLoadtype,FieldToper,FieldEoper,FieldOrigasn,FieldReasnby,FieldLocactlc,FieldLocacttp,FieldLocactrl,FieldAudit,FieldLsizetons,FieldDlock,FieldElock,FieldEdlock,FieldRlock,FieldTimearrivemobile,FieldTimedumpmobile,FieldTimeemptymobile,FieldPathtravtime,FieldExptraveldist,FieldExptraveltime,Tonnage,Truck,FieldEqmtTypeTruck,EqmtTypeTruck,Excav,FieldEqmtTypeExcav,EqmtTypeExcav,DumpLocation,FieldRegionDump,FieldUnitDump,DumpLocationUnit,BlastLocation,FieldRegionBlast,FieldUnitBlast,BlastLocationUnit,RegionBlast,RegionDump,MaterialType
0,210101002,2101010020001340352,1340352,2101010020000013616,2101010020000007908,2101010020000055856,2101010020001864544,2101010020000033264,2101010020000009052,-1,299,1798,1854,1893,246,90,0,1026,1036,412,2101010020000013868,2101010020000259400,NaN,NaN,NaN,0,NaN,-1,"298,8543",NINGUNA,PAB05,PRIM,NINGUNA,1609514997,1609515052,1609515091,175,1026,141,298.9,CDH41,135,KOM 930E,PAB05,154,BUCYRUS 495B/HR,PRIM,2101010020000003068,267,Chancador,STK_DONOSO_AL,2101010020000003068,268,Stock,MINA,MINA,Mineral
1,210101002,2101010020002075348,2075348,2101010020000013616,2101010020000007908,2101010020000055856,2101010020002428536,2101010020000033264,2101010020000009052,-1,297,2278,3054,3093,246,90,0,1026,1036,412,2101010020000013868,2101010020000259400,NaN,NaN,NaN,0,NaN,-1,"298,8543",NINGUNA,PAB05,PRIM,NINGUNA,1609515676,1609516253,1609516292,198,1026,146,298.9,CDH41,135,KOM 930E,PAB05,154,BUCYRUS 495B/HR,PRIM,2101010020000003068,267,Chancador,STK_DONOSO_AL,2101010020000003068,268,Stock,MINA,MINA,Mineral


##### Agregar tiempo descarga en formato epoch

In [ ]:
df = df_dumps.values.tolist()
new_column = []
for fila in df:
  turno =  fila[3]
  inicio_carga_seg = fila[12]
  año = str(turno)[0:2]
  mes = str(turno)[2:4]
  dia = str(turno)[4:6]
  #print(año,mes,dia)
  tipo_turno = str(turno)[7:9]
  if tipo_turno == '02':
    hora = '20:00:00'
  else:
    hora = '08:00:00'
  inicio_turno = time.mktime(time.strptime(dia+'/'+mes+'/'+año+' '+ hora,"%d/%m/%y %H:%M:%S"))
  inicio_carga = inicio_turno + inicio_carga_seg
  new_column.append(inicio_carga)

df_dumps['Timesec_start'] = new_column

In [ ]:
df_dumps = df_dumps.sort_values(["ShiftId",'Timesec_start'], ascending = ( True,True))

#### df_loads

In [ ]:
# DF LOADS
df_loads.head(2)

,ShiftId,Id,DbKey,FieldId,FieldTruck,FieldExcav,FieldGrade,FieldLoc,FieldDumprec,FieldTons,FieldTimearrive,FieldTimeload,FieldTimefull,FieldCalctravtime,FieldLoad,FieldExtraload,FieldLoadtype,FieldDist,FieldEfh,FieldToper,FieldEoper,FieldOrigasn,FieldReasnby,FieldLocactlc,FieldLocacttp,FieldLocactrl,FieldFirstdipper,FieldLastdipper,FieldBktcnt,FieldBktsum,FieldDumpasn,FieldLsizetons,FieldDlock,FieldElock,FieldEdlock,FieldRlock,FieldPathtravtime,FieldExptraveldist,FieldExptraveltime,Tonnage
0,210101002,2101010020000257488,257488,NaN,2101010020000031160,2101010020000033768,2101010020000057284,2101010020000006588,2101010020001873288,272,576,576,697,1653,90,0,412,8780,7946,2101010020000031412,2101010020000244252,NaN,NaN,NaN,0,NaN,1609542611,1609542683,3,2101010020000258612,2101010020000007996,"298,1546",NINGUNA,NINGUNA,PRIM DOS,NINGUNA,1228,5574,751,298.2
1,210101002,2101010020000265180,265180,NaN,2101010020000021828,2101010020000034132,2101010020000264792,2101010020000006236,2101010020001871384,335,758,758,854,1665,92,0,412,8879,8146,2101010020000022080,2101010020000256016,NaN,NaN,NaN,0,NaN,1609542795,1609542866,2,2101010020000437588,2101010020000006236,"303,0774",NINGUNA,NINGUNA,PRIM,NINGUNA,1212,6063,802,303.1


In [ ]:
df_loads['LoadingTime'] = df_loads['FieldTimefull'] - df_loads['FieldTimeload']

##### Agregar informacion Truck y Excav

In [ ]:
print('Filas antes de cruzar con eqmt:',len(df_loads))
df_loads = df_loads.drop(columns=['FieldId'])
df_loads = df_loads.merge(df_eqmt,left_on='FieldTruck', right_on='Id', how='inner')
print('Filas despues de cruzar con eqmt:',len(df_loads), 'No se deberia perder ninguna')
df_loads = df_loads.drop(columns=['Id_y'])
df_loads = df_loads.rename(columns={'Id_x':'Id'})
df_loads = df_loads.merge(df_enum, left_on='FieldEqmttype', right_on='Id', how='inner')
print('Filas despues de cruzar con enum:',len(df_loads), 'No se deberia perder ninguna')
df_loads = df_loads.drop(columns=['Id_y'])
df_loads = df_loads.rename(columns={'Id_x':'Id'})
df_loads = df_loads.rename(columns={'FieldEqmttype': 'FieldEqmtTypeTruck','Description':'EqmtTypeTruck', 'FieldIdEqmt':'Truck'})

Filas antes de cruzar con eqmt: 537186
Filas despues de cruzar con eqmt: 537186 No se deberia perder ninguna
Filas despues de cruzar con enum: 537186 No se deberia perder ninguna


In [ ]:
print('Filas antes de cruzar con eqmt:',len(df_loads))
df_loads = df_loads.merge(df_eqmt,left_on='FieldExcav', right_on='Id', how='inner')
print('Filas despues de cruzar con eqmt:',len(df_loads), 'No se deberia perder ninguna')
df_loads = df_loads.drop(columns=['Id_y'])
df_loads = df_loads.rename(columns={'Id_x':'Id'})
df_loads = df_loads.merge(df_enum, left_on='FieldEqmttype', right_on='Id', how='inner')
df_loads = df_loads.rename(columns={'FieldEqmttype': 'FieldEqmtTypeExcav','Description':'EqmtTypeExcav', 'FieldIdEqmt':'Excav'})
df_loads = df_loads.drop(columns=['Id_y'])
df_loads = df_loads.rename(columns={'Id_x':'Id'})
print('Filas despues de cruzar con enum:',len(df_loads), 'No se deberia perder ninguna')

Filas antes de cruzar con eqmt: 537186
Filas despues de cruzar con eqmt: 537186 No se deberia perder ninguna
Filas despues de cruzar con enum: 537186 No se deberia perder ninguna


##### Agregar información blast

In [ ]:
print('Filas antes de cruzar con shift_loc (blast):',len(df_loads))
df_loads = df_loads.merge(df_loc,  left_on='FieldLoc', right_on='Id',how='inner')
df_loads = df_loads.drop(columns=['Id_y'])
df_loads = df_loads.rename(columns={'Id_x':'Id'})
print('Filas despues de cruzar con shift_loc:',len(df_loads), 'No se deberia perder ninguna')
df_loads = df_loads.merge(df_enum, left_on='FieldUnit', right_on='Id', how='inner')
df_loads = df_loads.rename(columns={'Description':'BlastLocationUnit','FieldId':'BlastLocation','FieldRegion':'FieldRegionBlast','FieldUnit':'FieldUnitBlast'})
df_loads = df_loads.drop(columns=['Id_y'])
df_loads = df_loads.rename(columns={'Id_x':'Id'})
print('Filas despues de cruzar con enum:',len(df_loads), 'No se deberia perder ninguna')

Filas antes de cruzar con shift_loc (blast): 537186
Filas despues de cruzar con shift_loc: 537186 No se deberia perder ninguna
Filas despues de cruzar con enum: 537186 No se deberia perder ninguna


In [ ]:
print('Filas antes de cruzar con fases de df_loc (blast):',len(df_loads))
df_loads = df_loads.merge(df_fases,  left_on='FieldRegionBlast', right_on='Id_1',how='inner')
df_loads = df_loads.drop(columns=['Id_1'])
df_loads = df_loads.rename(columns={'fase':'RegionBlast'})
print('Filas despues de cruzar con df_loc:',len(df_loads), 'Solo se pierden filas si una fase no esta en shift_loc, podría ser por tener menos rango de fecha en shift_loc')

Filas antes de cruzar con fases de df_loc (blast): 537186
Filas despues de cruzar con df_loc: 498335 Solo se pierden filas si una fase no esta en shift_loc, podría ser por tener menos rango de fecha en shift_loc


##### Agregar tipo de material transportado

In [ ]:
##### para agregar tipo de material transportado
print('Filas antes de cruzar con enum:',len(df_loads))
df_loads = df_loads.merge(df_enum,  left_on='FieldLoad', right_on='Id',how='inner')
print('Filas despues de cruzar con enum:',len(df_loads), 'No deberia perder filas')
df_loads = df_loads.drop(columns=['Id_y'])
df_loads = df_loads.rename(columns={'Description':'MaterialType','Id_x':'Id'})

Filas antes de cruzar con enum: 498335
Filas despues de cruzar con enum: 498335 No deberia perder filas


##### Se agrega tiempo de inicio de descarga en formato epoch 

In [ ]:
df = df_loads.values.tolist()
new_column = []
for fila in df:
  turno =  fila[3]
  inicio_carga_seg = fila[10]
  año = str(turno)[0:2]
  mes = str(turno)[2:4]
  dia = str(turno)[4:6]
  #print(año,mes,dia)
  tipo_turno = str(turno)[7:9]
  if tipo_turno == '02':
    hora = '20:00:00'
  else:
    hora = '08:00:00'
  inicio_turno = time.mktime(time.strptime(dia+'/'+mes+'/'+año+' '+ hora,"%d/%m/%y %H:%M:%S"))
  inicio_carga = inicio_turno + inicio_carga_seg
  new_column.append(inicio_carga)

df_loads['Timesec_start'] = new_column

In [ ]:
df_loads = df_loads.sort_values(["ShiftId",'Timesec_start'], ascending = ( True,True))

#Nuevo : Cruzar con dumps para obtener destino y otros (Se necesita para obtener indisponibilidades simulador formato Json)

In [ ]:
def merge_dataframes(df1, df2, left_field , right_field, df2_tag, how_merge = 'inner'):
  print("len before merge:", df1.shape[0])
  columns_1 = list(df1.columns)
  columns_2 = list(df2.columns)
  #df = df1.merge(df2, left_on=left_field, right_on=right_field, how= 'left')
  df = df1.merge(df2, left_on=left_field, right_on=right_field, how= how_merge)
  df.columns = columns_1 + [name + df2_tag for name in columns_2]
  print("len after merge:", df.shape[0])
  print("")
  return df

def format_datetime(str_datetime):
  yy, mm, dd = map(int,str_datetime.split(' ')[0].split('-'))
  HH, MM, SS = map(int,str_datetime.split(' ')[1].split('.')[0].split(':'))
  dt = datetime(yy,mm,dd,HH,MM,SS)
  return dt

#df_traveltimes_loaded = merge_dataframes(dfv_loads_cruce, dfv_dumps_cruce, "NextShiftDumpId","Id", "_dumps", 'inner')
df_traveltimes_loaded = merge_dataframes(df_loads, df_dumps, "FieldDumprec","Id", "_dumps", 'left')

# REVISAR MISSMATCH (Truck, excav, blast)
df_traveltimes_loaded['match_key_loads'] = \
            df_traveltimes_loaded.Truck.astype(str) +"#" + df_traveltimes_loaded.Excav.astype(str) + "#" + df_traveltimes_loaded.BlastLocation
df_traveltimes_loaded['match_key_dumps'] = \
            df_traveltimes_loaded.Truck_dumps.astype(str) +"#"+ df_traveltimes_loaded.Excav_dumps.astype(str)  + "#" + + df_traveltimes_loaded.BlastLocation_dumps

print("missmatch (truck, excav, blast):", df_traveltimes_loaded[df_traveltimes_loaded.match_key_dumps != 
                                                       df_traveltimes_loaded.match_key_loads].shape[0])

# Dejar solo aquellos viajes en donde coincide toda la "match_key"
df_traveltimes_loaded = \
  df_traveltimes_loaded[df_traveltimes_loaded.match_key_loads == df_traveltimes_loaded.match_key_dumps].reset_index(drop = True)

# Formatear tiempos a datetime
#df_traveltimes_loaded['dt_arrive_dumps'] = df_traveltimes_loaded.FieldTimearrive_dumps.apply(format_datetime)
#df_traveltimes_loaded['dt_full_load'] = df_traveltimes_loaded.FieldTimefull.apply(format_datetime)
#df_traveltimes_loaded['dt_arrive_dump'] = df_traveltimes_loaded.FieldTimearrive_dumps
#df_traveltimes_loaded['dt_full_load'] = df_traveltimes_loaded.FieldTimefull

df_traveltimes_loaded['travel_time']= (df_traveltimes_loaded.FieldTimearrive_dumps - df_traveltimes_loaded.FieldTimefull)   # Tiempo de viaje lleno

df_loads = df_traveltimes_loaded

len before merge: 498335
len after merge: 498335

missmatch (truck, excav, blast): 4332


# Crear df_loads, especial para analisis power bi

In [ ]:
df_loads.columns.tolist()

['ShiftId',
 'Id',
 'DbKey',
 'FieldTruck',
 'FieldExcav',
 'FieldGrade',
 'FieldLoc',
 'FieldDumprec',
 'FieldTons',
 'FieldTimearrive',
 'FieldTimeload',
 'FieldTimefull',
 'FieldCalctravtime',
 'FieldLoad',
 'FieldExtraload',
 'FieldLoadtype',
 'FieldDist',
 'FieldEfh',
 'FieldToper',
 'FieldEoper',
 'FieldOrigasn',
 'FieldReasnby',
 'FieldLocactlc',
 'FieldLocacttp',
 'FieldLocactrl',
 'FieldFirstdipper',
 'FieldLastdipper',
 'FieldBktcnt',
 'FieldBktsum',
 'FieldDumpasn',
 'FieldLsizetons',
 'FieldDlock',
 'FieldElock',
 'FieldEdlock',
 'FieldRlock',
 'FieldPathtravtime',
 'FieldExptraveldist',
 'FieldExptraveltime',
 'Tonnage',
 'LoadingTime',
 'Truck',
 'FieldEqmtTypeTruck',
 'EqmtTypeTruck',
 'Excav',
 'FieldEqmtTypeExcav',
 'EqmtTypeExcav',
 'BlastLocation',
 'FieldRegionBlast',
 'FieldUnitBlast',
 'BlastLocationUnit',
 'RegionBlast',
 'MaterialType',
 'Timesec_start',
 'ShiftId_dumps',
 'Id_dumps',
 'DbKey_dumps',
 'FieldTruck_dumps',
 'FieldLoc_dumps',
 'FieldGrade_dumps',
 

In [ ]:
df_loads

,ShiftId,Id,DbKey,FieldTruck,FieldExcav,FieldGrade,FieldLoc,FieldDumprec,FieldTons,FieldTimearrive,FieldTimeload,FieldTimefull,FieldCalctravtime,FieldLoad,FieldExtraload,FieldLoadtype,FieldDist,FieldEfh,FieldToper,FieldEoper,FieldOrigasn,FieldReasnby,FieldLocactlc,FieldLocacttp,FieldLocactrl,FieldFirstdipper,FieldLastdipper,FieldBktcnt,FieldBktsum,FieldDumpasn,FieldLsizetons,FieldDlock,FieldElock,FieldEdlock,FieldRlock,FieldPathtravtime,FieldExptraveldist,FieldExptraveltime,Tonnage,LoadingTime,Truck,FieldEqmtTypeTruck,EqmtTypeTruck,Excav,FieldEqmtTypeExcav,EqmtTypeExcav,BlastLocation,FieldRegionBlast,FieldUnitBlast,BlastLocationUnit,RegionBlast,MaterialType,Timesec_start,ShiftId_dumps,Id_dumps,DbKey_dumps,FieldTruck_dumps,FieldLoc_dumps,FieldGrade_dumps,FieldLoadrec_dumps,FieldExcav_dumps,FieldBlast_dumps,FieldBay_dumps,FieldTons_dumps,FieldTimearrive_dumps,FieldTimedump_dumps,FieldTimeempty_dumps,FieldCalctravtime_dumps,FieldLoad_dumps,FieldExtraload_dumps,FieldDist_dumps,FieldEfh_dumps,FieldLoadtype_dumps,FieldToper_dumps,FieldEoper_dumps,FieldOrigasn_dumps,FieldReasnby_dumps,FieldLocactlc_dumps,FieldLocacttp_dumps,FieldLocactrl_dumps,FieldAudit_dumps,FieldLsizetons_dumps,FieldDlock_dumps,FieldElock_dumps,FieldEdlock_dumps,FieldRlock_dumps,FieldTimearrivemobile_dumps,FieldTimedumpmobile_dumps,FieldTimeemptymobile_dumps,FieldPathtravtime_dumps,FieldExptraveldist_dumps,FieldExptraveltime_dumps,Tonnage_dumps,Truck_dumps,FieldEqmtTypeTruck_dumps,EqmtTypeTruck_dumps,Excav_dumps,FieldEqmtTypeExcav_dumps,EqmtTypeExcav_dumps,DumpLocation_dumps,FieldRegionDump_dumps,FieldUnitDump_dumps,DumpLocationUnit_dumps,BlastLocation_dumps,FieldRegionBlast_dumps,FieldUnitBlast_dumps,BlastLocationUnit_dumps,RegionBlast_dumps,RegionDump_dumps,MaterialType_dumps,Timesec_start_dumps,match_key_loads,match_key_dumps,travel_time
0,210101002,2101010020000257488,257488,2101010020000031160,2101010020000033768,2101010020000057284,2101010020000006588,2101010020001873288,272,576,576,697,1653,90,0,412,8780,7946,2101010020000031412,2101010020000244252,NaN,NaN,NaN,0,NaN,1609542611,1609542683,3,2101010020000258612,2101010020000007996,"298,1546",NINGUNA,NINGUNA,PRIM DOS,NINGUNA,1228,5574,751,298.2,121,CDH91,135,KOM 930E,PAB08,154,BUCYRUS 495B/HR,IS5-3475-532,2101010020000005180,269,Banco,INFIERNILLO 5,Mineral,1609531776.0,210101002.0,2101010020001873408.0,1873288.0,2101010020000031232.0,2101010020000007936.0,2101010020000057344.0,2101010020002427392.0,2101010020000033792.0,2101010020000006656.0,-1.0,272,1772.0,2270.0,2327.0,1374.0,90.0,0.0,5525.0,5814.0,412.0,2101010020000031488.0,2101010020000244224.0,NaN,NaN,NaN,0.0,NaN,-1.0,"298,1546",NINGUNA,NINGUNA,PRIM DOS,NINGUNA,1609532969.0,1609533470.0,1609533525.0,991.0,5525.0,847.0,298.2,CDH91,135.0,KOM 930E,PAB08,154.0,BUCYRUS 495B/HR,PRIM DOS,2101010020000003072.0,267.0,Chancador,IS5-3475-532,2101010020000005120.0,269.0,Banco,INFIERNILLO 5,MINA,Mineral,1609533470.0,CDH91#PAB08#IS5-3475-532,CDH91#PAB08#IS5-3475-532,1075.0
1,210101002,2101010020000265180,265180,2101010020000021828,2101010020000034132,2101010020000264792,2101010020000006236,2101010020001871384,335,758,758,854,1665,92,0,412,8879,8146,2101010020000022080,2101010020000256016,NaN,NaN,NaN,0,NaN,1609542795,1609542866,2,2101010020000437588,2101010020000006236,"303,0774",NINGUNA,NINGUNA,PRIM,NINGUNA,1212,6063,802,303.1,96,CDH64,135,KOM 930E,PAB10,154,BUCYRUS 495B/HR,IS5-3475-512,2101010020000005180,269,Banco,INFIERNILLO 5,Esteril,1609531958.0,210101002.0,2101010020001871360.0,1871384.0,2101010020000021760.0,2101010020000003584.0,2101010020000264704.0,2101010020002435072.0,2101010020000034048.0,2101010020000006144.0,-1.0,0,2179.0,2217.0,2288.0,1594.0,92.0,0.0,6757.0,6892.0,412.0,2101010020000022016.0,2101010020000256000.0,NaN,NaN,NaN,0.0,NaN,2101010020032728832.0,"303,0774",NINGUNA,NINGUNA,PRIM,NINGUNA,1609533378.0,1609533416.0,1609533486.0,1185.0,6757.0,1064.0,303.1,CDH64,135.0,KOM 930E,PAB10,154.0,BUCYRUS 495B/HR,BOT-CURVALORO/ESTERIL,2101010020000003072.0,26

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
print(df_loads.columns.tolist())

['ShiftId', 'Id', 'DbKey', 'FieldTruck', 'FieldExcav', 'FieldGrade', 'FieldLoc', 'FieldDumprec', 'FieldTons', 'FieldTimearrive', 'FieldTimeload', 'FieldTimefull', 'FieldCalctravtime', 'FieldLoad', 'FieldExtraload', 'FieldLoadtype', 'FieldDist', 'FieldEfh', 'FieldToper', 'FieldEoper', 'FieldOrigasn', 'FieldReasnby', 'FieldLocactlc', 'FieldLocacttp', 'FieldLocactrl', 'FieldFirstdipper', 'FieldLastdipper', 'FieldBktcnt', 'FieldBktsum', 'FieldDumpasn', 'FieldLsizetons', 'FieldDlock', 'FieldElock', 'FieldEdlock', 'FieldRlock', 'FieldPathtravtime', 'FieldExptraveldist', 'FieldExptraveltime', 'Tonnage', 'LoadingTime', 'Truck', 'FieldEqmtTypeTruck', 'EqmtTypeTruck', 'Excav', 'FieldEqmtTypeExcav', 'EqmtTypeExcav', 'BlastLocation', 'FieldRegionBlast', 'FieldUnitBlast', 'BlastLocationUnit', 'RegionBlast', 'MaterialType', 'Timesec_start', 'ShiftId_dumps', 'Id_dumps', 'DbKey_dumps', 'FieldTruck_dumps', 'FieldLoc_dumps', 'FieldGrade_dumps', 'FieldLoadrec_dumps', 'FieldExcav_dumps', 'FieldBlast_dumps

In [ ]:
df_loads[0:5]

,ShiftId,Id,DbKey,FieldTruck,FieldExcav,FieldGrade,FieldLoc,FieldDumprec,FieldTons,FieldTimearrive,FieldTimeload,FieldTimefull,FieldCalctravtime,FieldLoad,FieldExtraload,FieldLoadtype,FieldDist,FieldEfh,FieldToper,FieldEoper,FieldOrigasn,FieldReasnby,FieldLocactlc,FieldLocacttp,FieldLocactrl,FieldFirstdipper,FieldLastdipper,FieldBktcnt,FieldBktsum,FieldDumpasn,FieldLsizetons,FieldDlock,FieldElock,FieldEdlock,FieldRlock,FieldPathtravtime,FieldExptraveldist,FieldExptraveltime,Tonnage,LoadingTime,Truck,FieldEqmtTypeTruck,EqmtTypeTruck,Excav,FieldEqmtTypeExcav,EqmtTypeExcav,BlastLocation,FieldRegionBlast,FieldUnitBlast,BlastLocationUnit,RegionBlast,MaterialType,Timesec_start,ShiftId_dumps,Id_dumps,DbKey_dumps,FieldTruck_dumps,FieldLoc_dumps,FieldGrade_dumps,FieldLoadrec_dumps,FieldExcav_dumps,FieldBlast_dumps,FieldBay_dumps,FieldTons_dumps,FieldTimearrive_dumps,FieldTimedump_dumps,FieldTimeempty_dumps,FieldCalctravtime_dumps,FieldLoad_dumps,FieldExtraload_dumps,FieldDist_dumps,FieldEfh_dumps,FieldLoadtype_dumps,FieldToper_dumps,FieldEoper_dumps,FieldOrigasn_dumps,FieldReasnby_dumps,FieldLocactlc_dumps,FieldLocacttp_dumps,FieldLocactrl_dumps,FieldAudit_dumps,FieldLsizetons_dumps,FieldDlock_dumps,FieldElock_dumps,FieldEdlock_dumps,FieldRlock_dumps,FieldTimearrivemobile_dumps,FieldTimedumpmobile_dumps,FieldTimeemptymobile_dumps,FieldPathtravtime_dumps,FieldExptraveldist_dumps,FieldExptraveltime_dumps,Tonnage_dumps,Truck_dumps,FieldEqmtTypeTruck_dumps,EqmtTypeTruck_dumps,Excav_dumps,FieldEqmtTypeExcav_dumps,EqmtTypeExcav_dumps,DumpLocation_dumps,FieldRegionDump_dumps,FieldUnitDump_dumps,DumpLocationUnit_dumps,BlastLocation_dumps,FieldRegionBlast_dumps,FieldUnitBlast_dumps,BlastLocationUnit_dumps,RegionBlast_dumps,RegionDump_dumps,MaterialType_dumps,Timesec_start_dumps,match_key_loads,match_key_dumps,travel_time
0,210101002,2101010020000257488,257488,2101010020000031160,2101010020000033768,2101010020000057284,2101010020000006588,2101010020001873288,272,576,576,697,1653,90,0,412,8780,7946,2101010020000031412,2101010020000244252,NaN,NaN,NaN,0,NaN,1609542611,1609542683,3,2101010020000258612,2101010020000007996,"298,1546",NINGUNA,NINGUNA,PRIM DOS,NINGUNA,1228,5574,751,298.2,121,CDH91,135,KOM 930E,PAB08,154,BUCYRUS 495B/HR,IS5-3475-532,2101010020000005180,269,Banco,INFIERNILLO 5,Mineral,1609531776.0,210101002.0,2101010020001873408.0,1873288.0,2101010020000031232.0,2101010020000007936.0,2101010020000057344.0,2101010020002427392.0,2101010020000033792.0,2101010020000006656.0,-1.0,272,1772.0,2270.0,2327.0,1374.0,90.0,0.0,5525.0,5814.0,412.0,2101010020000031488.0,2101010020000244224.0,NaN,NaN,NaN,0.0,NaN,-1.0,"298,1546",NINGUNA,NINGUNA,PRIM DOS,NINGUNA,1609532969.0,1609533470.0,1609533525.0,991.0,5525.0,847.0,298.2,CDH91,135.0,KOM 930E,PAB08,154.0,BUCYRUS 495B/HR,PRIM DOS,2101010020000003072.0,267.0,Chancador,IS5-3475-532,2101010020000005120.0,269.0,Banco,INFIERNILLO 5,MINA,Mineral,1609533470.0,CDH91#PAB08#IS5-3475-532,CDH91#PAB08#IS5-3475-532,1075.0
1,210101002,2101010020000265180,265180,2101010020000021828,2101010020000034132,2101010020000264792,2101010020000006236,2101010020001871384,335,758,758,854,1665,92,0,412,8879,8146,2101010020000022080,2101010020000256016,NaN,NaN,NaN,0,NaN,1609542795,1609542866,2,2101010020000437588,2101010020000006236,"303,0774",NINGUNA,NINGUNA,PRIM,NINGUNA,1212,6063,802,303.1,96,CDH64,135,KOM 930E,PAB10,154,BUCYRUS 495B/HR,IS5-3475-512,2101010020000005180,269,Banco,INFIERNILLO 5,Esteril,1609531958.0,210101002.0,2101010020001871360.0,1871384.0,2101010020000021760.0,2101010020000003584.0,2101010020000264704.0,2101010020002435072.0,2101010020000034048.0,2101010020000006144.0,-1.0,0,2179.0,2217.0,2288.0,1594.0,92.0,0.0,6757.0,6892.0,412.0,2101010020000022016.0,2101010020000256000.0,NaN,NaN,NaN,0.0,NaN,2101010020032728832.0,"303,0774",NINGUNA,NINGUNA,PRIM,NINGUNA,1609533378.0,1609533416.0,1609533486.0,1185.0,6757.0,1064.0,303.1,CDH64,135.0,KOM 930E,PAB10,154.0,BUCYRUS 495B/HR,BOT-CURVALORO/ESTERIL,2101010020000003072.0,26

In [ ]:
df_loads_analisis_Pbi = df_loads[['ShiftId', 'Excav','EqmtTypeExcav', 'Truck','EqmtTypeTruck','Tonnage','BlastLocation', 'RegionBlast','DumpLocation_dumps','DumpLocationUnit_dumps','MaterialType','FieldTimearrive_dumps','FieldTimedump_dumps','FieldTimeempty_dumps','FieldTimeload','FieldTimearrive','FieldTimefull','FieldPathtravtime','FieldPathtravtime_dumps', 'Timesec_start']]

In [ ]:
df_loads_analisis_Pbi = df_loads_analisis_Pbi.rename(columns={'DumpLocation_dumps':'DumpLocation','DumpLocationUnit_dumps':'DumpLocationUnit','FieldTimedump_dumps':'FieldTimedump'})
df_loads_analisis_Pbi = df_loads_analisis_Pbi.rename(columns={'FieldTimearrive_dumps':'FieldTimearrivedump','FieldTimeempty_dumps':'FieldTimeempty','FieldTimearrive':'FieldTimearriveload'})
df_loads_analisis_Pbi = df_loads_analisis_Pbi.rename(columns={'FieldPathtravtime':'FieldPathtravtimeempty','FieldPathtravtime_dumps':'FieldPathtravtime', 'Timesec_start':'dt_load'})

In [ ]:
df_loads_analisis_Pbi = df_loads_analisis_Pbi.rename(columns={'FieldPathtravtimeloaded':'FieldPathtravtime'})

In [ ]:
df_loads_analisis_Pbi['Queuetimeload']= (df_loads_analisis_Pbi.FieldTimeload - df_loads_analisis_Pbi.FieldTimearriveload)
df_loads_analisis_Pbi['Queuetimedump']= (df_loads_analisis_Pbi.FieldTimedump - df_loads_analisis_Pbi.FieldTimearrivedump)
df_loads_analisis_Pbi['LoadingTime']= (df_loads_analisis_Pbi.FieldTimefull - df_loads_analisis_Pbi.FieldTimeload)
df_loads_analisis_Pbi['UnloadingTime']= (df_loads_analisis_Pbi.FieldTimeempty - df_loads_analisis_Pbi.FieldTimedump)

In [ ]:
df_loads_analisis_Pbi[0:2]

,ShiftId,Excav,EqmtTypeExcav,Truck,EqmtTypeTruck,Tonnage,BlastLocation,RegionBlast,DumpLocation,DumpLocationUnit,MaterialType,FieldTimearrivedump,FieldTimedump,FieldTimeempty,FieldTimeload,FieldTimearriveload,FieldTimefull,FieldPathtravtimeempty,FieldPathtravtime,dt_load,Queuetimeload,Queuetimedump,LoadingTime,UnloadingTime
0,210101002,PAB08,BUCYRUS 495B/HR,CDH91,KOM 930E,298.2,IS5-3475-532,INFIERNILLO 5,PRIM DOS,Chancador,Mineral,1772.0,2270.0,2327.0,576,576,697,1228,991.0,1609531776.0,0,498.0,121,57.0
1,210101002,PAB10,BUCYRUS 495B/HR,CDH64,KOM 930E,303.1,IS5-3475-512,INFIERNILLO 5,BOT-CURVALORO/ESTERIL,Botadero,Esteril,2179.0,2217.0,2288.0,758,758,854,1212,1185.0,1609531958.0,0,38.0,96,71.0


# Descargar

# Crear df_loads, especial para simulador y Pbi, estandarización entidades
*texto en cursiva*

In [ ]:
df_loads_simulador = df_loads[['ShiftId', 'Excav','EqmtTypeExcav', 'Truck','EqmtTypeTruck','Tonnage','BlastLocation', 'RegionBlast','DumpLocation_dumps','DumpLocationUnit_dumps','MaterialType','FieldTimearrive_dumps','FieldTimedump_dumps','FieldTimeempty_dumps','FieldTimeload','FieldTimearrive','FieldTimefull','FieldPathtravtime','FieldPathtravtime_dumps', 'Timesec_start']]

In [ ]:
df_loads_simulador[0:5]

,ShiftId,Excav,EqmtTypeExcav,Truck,EqmtTypeTruck,Tonnage,BlastLocation,RegionBlast,DumpLocation_dumps,DumpLocationUnit_dumps,MaterialType,FieldTimearrive_dumps,FieldTimedump_dumps,FieldTimeempty_dumps,FieldTimeload,FieldTimearrive,FieldTimefull,FieldPathtravtime,FieldPathtravtime_dumps,Timesec_start
0,210101002,PAB08,BUCYRUS 495B/HR,CDH91,KOM 930E,298.2,IS5-3475-532,INFIERNILLO 5,PRIM DOS,Chancador,Mineral,1772.0,2270.0,2327.0,576,576,697,1228,991.0,1609531776.0
1,210101002,PAB10,BUCYRUS 495B/HR,CDH64,KOM 930E,303.1,IS5-3475-512,INFIERNILLO 5,BOT-CURVALORO/ESTERIL,Botadero,Esteril,2179.0,2217.0,2288.0,758,758,854,1212,1185.0,1609531958.0
2,210101002,PAB08,BUCYRUS 495B/HR,CDH81,KOM 930E,298.2,IS5-3475-532,INFIERNILLO 5,PRIM,Chancador,Mineral,2070.0,2150.0,2207.0,772,487,864,1230,1075.0,1609531972.0
3,210101002,PAB08,BUCYRUS 495B/HR,CDH61,KOM 930E,298.2,IS5-3475-532,INFIERNILLO 5,PRIM,Chancador,Mineral,2146.0,2576.0,2660.0,936,712,1036,1131,1066.0,1609532136.0
4,210101002,PAB10,BUCYRUS 495B/HR,CDH53,KOM 930E,303.1,IS5-3475-512,INFIERNILLO 5,BOT-CURVALORO/ESTERIL,Botadero,Esteril,2378.0,2415.0,2474.0,955,630,1023,955,1162.0,1609532155.0


In [ ]:
df_loads_simulador = df_loads_simulador.rename(columns={'DumpLocation_dumps':'DumpLocation','DumpLocationUnit_dumps':'DumpLocationUnit','FieldTimedump_dumps':'FieldTimedump'})
df_loads_simulador = df_loads_simulador.rename(columns={'FieldTimearrive_dumps':'FieldTimearrivedump','FieldTimeempty_dumps':'FieldTimeempty','FieldTimearrive':'FieldTimearriveload'})
df_loads_simulador = df_loads_simulador.rename(columns={'FieldPathtravtime':'FieldPathtravtimeempty','FieldPathtravtime_dumps':'FieldPathtravtimeloaded', 'Timesec_start':'Timeload_epoch'})

In [ ]:
df_loads_simulador['Queuetimeload']= (df_loads_simulador.FieldTimeload - df_loads_simulador.FieldTimearriveload)
df_loads_simulador['Queuetimedump']= (df_loads_simulador.FieldTimedump - df_loads_simulador.FieldTimearrivedump)
df_loads_simulador['LoadingTime']= (df_loads_simulador.FieldTimefull - df_loads_simulador.FieldTimeload)
df_loads_simulador['UnloadingTime']= (df_loads_simulador.FieldTimeempty - df_loads_simulador.FieldTimedump)

## Prefijo Excav

In [ ]:
import re
def get_number(eqmt):
  try:
    match = re.match(r"([a-z]+)([0-9]+)", eqmt, re.I)
    if match:
        items = match.groups()

    return items[1]
  except:
    return str(eqmt)

In [ ]:
df_loads_simulador['EqmtTypeExcav'].unique().tolist()
df_loads_simulador['Excav'].unique().tolist()
# df_loads_simulador['Excav_new'].unique().tolist()

['PAB08',
 'PAB10',
 'PHK15',
 'PAB12',
 'PAB05',
 'PHK09',
 'PHK13',
 'PAB06',
 'PAB11',
 'VEC01',
 'CFL08',
 'PHK14',
 'CRA01',
 'VIR01']

In [ ]:
df_loads_simulador.groupby("EqmtTypeExcav").agg({"Tonnage": "mean"})['Tonnage']

EqmtTypeExcav
BUCYRUS 495B/HR   305.7
KOM PC 5500       303.9
LT 1850           304.9
VECCHIOLA_CARG    301.8
Name: Tonnage, dtype: float64

In [ ]:
dic_excav_type_size = {}  # hardcodeado
dic_excav_type_size['BUCYRUS 495B/HR'] = 'B'
dic_excav_type_size['VECCHIOLA_CARG'] = 'Ext'
dic_excav_type_size['KOM PC 5500'] = 'M'
dic_excav_type_size['LT 1850'] = 'S'

# AGREGAR OTRAS PALAS EXTERNAS   # hardcodeado
EXTERNAL_SHOVELS = ["VIR01", "CRA01", "VEC01"]

In [ ]:
df_loads_simulador_list = df_loads_simulador.values.tolist()
new_column = []
dic_excav = {}
for fila in df_loads_simulador_list:
  excav = fila[1]
  excav_type = fila[2]
  excav_size = dic_excav_type_size[excav_type]
  excav_name = 'P'+ excav_size + get_number(excav)
  if excav in EXTERNAL_SHOVELS:
    excav_name = 'P' + 'Ext' + excav
  new_column.append(excav_name)
  dic_excav[excav_name] = excav

df_loads_simulador['Excav_new'] = new_column

## Prefijo Chancadores

In [ ]:
df_loads_simulador['DumpLocationUnit'].unique().tolist()

['Chancador', 'Botadero', 'Stock']

In [ ]:
df_loads_simulador_chanc = df_loads_simulador[df_loads_simulador['DumpLocationUnit']=='Chancador']
df_loads_simulador_chanc['DumpLocation'].unique().tolist()

['PRIM DOS', 'PRIM']

In [ ]:
dic_chanc = {}  # hardcodeado
dic_chanc['PRIM'] = 'CHANC-1'
dic_chanc['PRIM DOS'] = 'CHANC-2'

In [ ]:
df_loads_simulador_list = df_loads_simulador.values.tolist()
new_column = []
for fila in df_loads_simulador_list:
  destino = fila[8]
  if destino in dic_chanc.keys():
    fila[8] = dic_chanc[destino]

columns = df_loads_simulador.columns
df_loads_simulador = pd.DataFrame(df_loads_simulador_list, columns=columns)

## Prefijo Stocks

In [ ]:
df_loads_simulador_stock = df_loads_simulador[df_loads_simulador['DumpLocationUnit']=='Stock']
stocks = df_loads_simulador_stock['DumpLocation'].unique().tolist()

In [ ]:
# Ahora se le pone STOCK_ a todos los stocks
dic_stocks = {}
for stock in stocks:
  stock_new = 'STOCK_' + stock
  dic_stocks[stock] = stock_new


In [ ]:
df_loads_simulador_list = df_loads_simulador.values.tolist()
new_column = []
for fila in df_loads_simulador_list:
  destino = fila[8]
  if destino in dic_stocks.keys():
    fila[8] = dic_stocks[destino]

columns = df_loads_simulador.columns
df_loads_simulador = pd.DataFrame(df_loads_simulador_list, columns=columns)

In [ ]:
df_loads_simulador_stock['DumpLocation'].unique().tolist()

['STK_DONOSO_ML',
 'STK_3600_ML',
 'STK_DONOSO_AL',
 'STK_SFCO_AL',
 'STK_SFCO_ML',
 'STK_ENAEX_BOL',
 'STK_ENAEX_AL',
 'STK_BOLONES_3600ML',
 'STK_3600_3620',
 'STK_PARRILLAS',
 'STK_SFCO_AD',
 'STK_3600_PRIMARIO',
 'STK_SOBRECARGAMINERAL',
 'STK_DON_AL',
 'STK_DON_ML',
 'STK_DON_3537',
 'STK_DON_3550_AL',
 'STOCK-ESTERIL-DON 2',
 'STK_3600_BOL',
 'STK_3600_3635',
 'STK_ALTOARSENICO_3600',
 'STK_SFCO_RAMPA',
 'STK_3600_3635_FLEXI',
 'STK_SFCO_BOLONES',
 'STK_DON_3550',
 'STK_3600_3635_FLEXI_B',
 'STK_3600_3635_FLEXI_A',
 'STK-4090-INF6',
 'STK_SFCO_3565',
 'STK_3600_3615-1',
 'STK_3600_3615',
 'STK_3600_W04',
 'STK_3370_L-8']

## Prefijo Dumps

In [ ]:
df_loads_simulador_dumps = df_loads_simulador[df_loads_simulador['DumpLocationUnit']=='Botadero']
dumps = df_loads_simulador_dumps['DumpLocation'].unique().tolist()

In [ ]:
# cambio de planes, ahora se le pone STOCK_ a todos los stocks
dic_dumps = {}
for dump in dumps:
  dump_new = 'BOT_' + dump
  dic_dumps[dump] = dump_new


In [ ]:
df_loads_simulador_list = df_loads_simulador.values.tolist()
new_column = []
for fila in df_loads_simulador_list:
  destino = fila[8]
  if destino in dic_dumps.keys():
    fila[8] = dic_dumps[destino]

columns = df_loads_simulador.columns
df_loads_simulador = pd.DataFrame(df_loads_simulador_list, columns=columns)

## Prefijo Trucks

In [ ]:
df_loads_simulador_list = df_loads_simulador.values.tolist()
new_column = []
dic_trucks = {}
list_truck_not_repeat = []
for fila in df_loads_simulador_list:
  truck = str(fila[3])
  # truck_type = fila[4]
  truck_name = 'T' + get_number(truck)
  if truck not in list_truck_not_repeat and truck_name not in dic_trucks.keys():
    new_column.append(truck_name)
    dic_trucks[truck_name] = truck
    list_truck_not_repeat.append(truck)
  elif truck not in list_truck_not_repeat:
    truck_name = truck_name + '0'
    new_column.append(truck_name)
    dic_trucks[truck_name] = truck
    list_truck_not_repeat.append(truck)
  else:
    new_column.append(truck_name)


df_loads_simulador['Truck_new'] = new_column

## Ordenar

In [ ]:
df_loads_simulador = df_loads_simulador.drop(columns=['Truck', 'Excav'])
df_loads_simulador = df_loads_simulador.rename(columns={'Truck_new':'Truck','Excav_new':'Excav'})
df_loads_simulador = df_loads_simulador[['ShiftId', 'Excav','EqmtTypeExcav', 'Truck','EqmtTypeTruck','Tonnage','BlastLocation', 'RegionBlast','DumpLocation','DumpLocationUnit','MaterialType','FieldTimearriveload','FieldTimedump','FieldTimeempty','FieldTimeload','FieldTimearrivedump','FieldTimefull','FieldPathtravtimeempty','FieldPathtravtimeloaded', 'Timeload_epoch', 'Queuetimeload', 'Queuetimedump', 'LoadingTime', 'UnloadingTime']]

## maestro equipo

In [ ]:
dic_maestro = {}
for key in dic_chanc.keys():
  dic_maestro[dic_chanc[key]] = key
for key in dic_trucks.keys():
  dic_maestro[key] = dic_trucks[key]
for key in dic_dumps.keys():
  dic_maestro[dic_dumps[key]] = key
for key in dic_stocks.keys():
  dic_maestro[dic_stocks[key]] = key
for key in dic_excav.keys():
  dic_maestro[key] = dic_excav[key]

In [ ]:
list_maestro = []
for key in dic_chanc.keys():
  list_maestro.append([key,  dic_chanc[key], 'Crusher'])
for key in dic_trucks.keys():
  list_maestro.append([key,  dic_trucks[key], 'Truck'])
for key in dic_dumps.keys():
  list_maestro.append([dic_dumps[key],  key, 'Dump'])
for key in dic_stocks.keys():
  list_maestro.append([dic_stocks[key],  key, 'Stock'])
for key in dic_excav.keys():
  list_maestro.append([key,  dic_excav[key], 'Excav'])

# Para Shiftshiftstate

In [ ]:
df_state = pd.read_csv(f'{path_db}Shiftshiftstate.csv', sep=';') 
df_state = df_state[['FieldId','ShiftId','FieldEqmt','FieldAuxeqmt','FieldReason','FieldStatus']]

df_eqmt = pd.read_csv(f'{path_db}Shiftshifteqmt.csv',sep=';')
df_eqmt = df_eqmt[['Id','FieldId','FieldEqmttype','FieldUnit']].drop_duplicates()
df_eqmt = df_eqmt.rename(columns={'FieldId':'FieldIdEqmt','FieldUnit':'FieldEqmtTypeType'})


In [ ]:
df_aux = pd.read_csv(f'{path_db}Shiftshiftaux.csv', sep=';', encoding='latin-1', error_bad_lines=False)    #info chancado
df_aux = df_aux.merge(df_enum, left_on='FieldEqmttype', right_on='Id', how='inner')
df_aux = df_aux.rename(columns={'Id_x':'Id'})
df_aux['Description'].unique()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


array(['CAT 980G', 'CAT 992K', 'CAT 988H', 'FC 500 OP', 'CAT 773D',
       'KOM 785', 'Desconocido', 'CAT D10T', 'Otros', 'CAT 24H',
       'CAT 16M', 'PIT VIPER 271', 'CHANC PRIMARIO', 'CAT 533D',
       'CAT 385C', 'CAT D10R', 'CAT D11T', 'CAT 854G', 'CAT 854K',
       'KOM WD900', 'CAT 24M', 'CAT 395'], dtype=object)

In [ ]:
df_aux = df_aux[(df_aux['Description']=='CHANC PRIMARIO')]     #hardcodeado

In [ ]:
df_aux = df_aux[['Id','FieldId','FieldEqmttype','FieldUnit']]
df_aux = df_aux.rename(columns={'FieldId':'FieldIdEqmt','FieldUnit':'FieldEqmtTypeType'})

In [ ]:
# Agregar descripcion equipos
df_eqmt = pd.concat([df_eqmt, df_aux], axis=0, ignore_index=True)     #junta info chancado con palas
df_eqmt= df_eqmt.drop_duplicates()

In [ ]:
df_state[0:5]

,FieldId,ShiftId,FieldEqmt,FieldAuxeqmt,FieldReason,FieldStatus
0,00:00:34,210101002,2101010020000025316,-1,1000,248
1,00:00:44,210101002,2101010020000014992,-1,4500,249
2,00:00:49,210101002,2101010020000020484,-1,1000,248
3,00:01:06,210101002,-1,2101010020000053836,1582,248
4,00:01:09,210101002,2101010020000030180,-1,4360,249


In [ ]:
df2 = df_state.values.tolist()
df3 = []
for fila in df2:
    if float(fila[2]) < 0:
        fila[2] = fila[3]
    df3.append(fila)

df_state = pd.DataFrame(df3, columns =['FieldId','ShiftId','FieldEqmt','FieldAuxeqmt','FieldReason','FieldStatus'])
del df3
df_state = df_state.drop(columns=['FieldAuxeqmt'])


In [ ]:
print(len(df_state),'antes de cruzar con eqmt')
df_state = df_state.merge(df_eqmt, left_on='FieldEqmt', right_on='Id',how='inner')
df_state = df_state.drop(columns=['Id'])
print(len(df_state),'dp de cruzar con eqmt, deberia disminuir un poco, pq hay equipos auxiliares que no estan en eqmt')

1078020 antes de cruzar con eqmt
991163 dp de cruzar con eqmt, deberia disminuir un poco, pq hay equipos auxiliares que no estan en eqmt


In [ ]:
df_state = df_state.merge(df_enum, left_on='FieldEqmttype', right_on='Id',how='inner')
df_state = df_state.drop(columns=['Id'])
print(len(df_state),'dp de cruzar con enum, no deberia disminuir, solo disminuye si un tipo de equipo no esta en enum')
df_state = df_state.rename(columns={'Description':'Description_Excav'})

991163 dp de cruzar con enum, no deberia disminuir, solo disminuye si un tipo de equipo no esta en enum


In [ ]:
df_state[0:5]

,FieldId,ShiftId,FieldEqmt,FieldReason,FieldStatus,FieldIdEqmt,FieldEqmttype,FieldEqmtTypeType,Description_Excav
0,00:00:34,210101002,2101010020000025316,1000,248,CDH74,135,264,KOM 930E
1,00:01:31,210101002,2101010020000025316,4360,249,CDH74,135,264,KOM 930E
2,02:21:23,210101002,2101010020000025316,1000,248,CDH74,135,264,KOM 930E
3,03:01:32,210101002,2101010020000025316,3080,250,CDH74,135,264,KOM 930E
4,03:07:54,210101002,2101010020000025316,1000,248,CDH74,135,264,KOM 930E


In [ ]:
df_enum[0:5]

,Id,Description
0,1,*
1,2,Panne
2,3,Operativo
3,4,Reserva
4,5,Demora


In [ ]:

df_state = df_state.merge(df_enum, left_on='FieldEqmtTypeType', right_on='Id',how='inner')
df_state = df_state.drop(columns=['Id'])
print(len(df_state),'dp de cruzar con enum, no deberia disminuir, solo disminuye si un tipo de equipo no esta en enum')
df_state = df_state.rename(columns={'Description':'Eqmttype'})

def merge_dataframes(df1, df2, left_field , right_field, df2_tag, how_merge = 'inner'):
  print("len before merge:", df1.shape[0])
  columns_1 = list(df1.columns)
  columns_2 = list(df2.columns)
  #df = df1.merge(df2, left_on=left_field, right_on=right_field, how= 'left')
  df = df1.merge(df2, left_on=left_field, right_on=right_field, how= how_merge)
  df.columns = columns_1 + [name + df2_tag for name in columns_2]
  print("len after merge:", df.shape[0])
  print("")
  return df



991163 dp de cruzar con enum, no deberia disminuir, solo disminuye si un tipo de equipo no esta en enum


In [ ]:
df_state['Eqmttype'].unique()   # harcodeado

array(['Camion', 'Carguio', 'Sala Chancado'], dtype=object)

In [ ]:
# Filtro a palas y chancadores
#df_state = df_state[(df_state['Description_Excav']=='VECCHIOLA_CARG')|(df_state['Description_Excav']=='KOM PC 5500')|(df_state['Description_Excav']=='BUCYRUS 495B/HR')|(df_state['Description_Excav']=='LT 1850')|(df_state['Description_Excav']=='CHANC PRIMARIO')]
df_state = df_state[(df_state['Eqmttype']=='Carguio')|(df_state['Eqmttype']=='Sala Chancado')]    # hardcodeado

# Cruzar con reason
#df_reason = df_reason.merge(df_enum, left_on='FieldCategory', right_on='Id',how='left')
#df_reason = df_reason.rename(columns={'Description':'Description_category'})

# Agregar descripcion estado
print(len(df_state),'antes de cruzar con enum')
df_state = df_state.merge(df_enum, left_on='FieldStatus', right_on='Id',how='inner')
df_state = df_state.drop(columns=['Id'])
df_state = df_state.rename(columns={'Description':'Description_status'})



71568 antes de cruzar con enum


In [ ]:
df_state['Description_status'].unique()

array(['Operativo', 'Demora', 'Reserva', 'Panne'], dtype=object)

In [ ]:
# status reserva = operativo en chancadores
columns = df_state.columns.tolist()
df2 = df_state.values.tolist()
df3 = []
for fila in df2:
    eqmt_type = fila[9]
    estado = fila[10]
    if eqmt_type == 'Sala Chancado' and estado != 'Panne':    # hardcodeado
        fila[10] = 'Operativo'
    df3.append(fila)
df_state = pd.DataFrame(df3, columns =columns)
del df3

In [ ]:
df_state['Description_status'].fillna(df_state['Description_status'], inplace=True)
df_state["estado"] = [
        "Operativo" if x in ['Operativo', 'Demora'] else "No Operativo"    # hardcodeado
        for x in df_state["Description_status"]
    ]

# FORMATEAR AÑO, MES-------------------------------------------------------------------------------------------------------
df_state['Year'] = '20' + df_state.ShiftId.astype(str).str[0:2]
df_state['Month'] = df_state.ShiftId.astype(str).str[2:4]
df_state['Day'] = df_state.ShiftId.astype(str).str[4:6]
df_state['Shift_type'] = df_state.ShiftId.astype(str).str[-1]

df_states_merged = df_state
df_states_merged = df_states_merged.rename(columns={'FieldIdEqmt':'Name_eqmt', 'Description_Excav':'Eqmttype_eqmt'})

#df_excav_states_0 = df_states_merged[(df_states_merged.Unit_eqmt == 'Pala')|(df_states_merged.Unit_eqmt == 'Crusher')]
df_excav_states_0 = df_states_merged[['ShiftId', 'Year', 'Month', 'Day','Shift_type', 'FieldId', 'Name_eqmt', 'Eqmttype_eqmt', 'Description_status', 'estado']]
df_excav_states_0['HH'] = df_excav_states_0.FieldId.str[0:2]
df_excav_states_0['HH'] = df_excav_states_0.HH.astype(int)

# FILTRAR AQUELLOS MÁS GRANDES QUE EL LARGO DEL TURNO
print(df_excav_states_0.shape[0])
df_excav_states = df_excav_states_0[df_excav_states_0.HH < 12]
print(df_excav_states.shape[0])

# Formatear fechas

def get_shiftstart_time_hours(shift_type):
  if int(shift_type) == 1:
    return 8
  elif int(shift_type) == 2:
    return 20
  else:
    return "ERROR"

def get_datetime_time(df, field_time, field_year, field_month, field_day, field_shifttype):
  l_datetime_time = []
  for i in range(df.shape[0]):
    HH,MM,SS = map(int,df[field_time].iloc[i].split(":"))
    hh_shift_start = get_shiftstart_time_hours(df[field_shifttype].iloc[i])

    datetime_raw = datetime(int(df[field_year].iloc[i]), int(df[field_month].iloc[i]), int(df[field_day].iloc[i]),
                            HH,MM,SS)
    datetime_time = datetime_raw + timedelta(hours = hh_shift_start)
    l_datetime_time.append(datetime_time)

  return l_datetime_time

df_excav_states['dt_start'] = get_datetime_time(df_excav_states, "FieldId", "Year", "Month", "Day", "Shift_type")
                  
# Ordenar por equipo y fecha
df_excav_states = df_excav_states.sort_values(by=['Name_eqmt', 'dt_start'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


71568
71562


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df_excav_states[0:5]

,ShiftId,Year,Month,Day,Shift_type,FieldId,Name_eqmt,Eqmttype_eqmt,Description_status,estado,HH,dt_start
56434,210103002,2021,01,03,2,00:49:52,CFL08,LT 1850,Demora,Operativo,0,2021-01-03 20:49:52
66315,210103002,2021,01,03,2,03:43:12,CFL08,LT 1850,Reserva,No Operativo,3,2021-01-03 23:43:12
26032,210104001,2021,01,04,1,02:01:57,CFL08,LT 1850,Operativo,Operativo,2,2021-01-04 10:01:57
56435,210104001,2021,01,04,1,11:34:31,CFL08,LT 1850,Demora,Operativo,11,2021-01-04 19:34:31
26033,210104002,2021,01,04,2,00:42:36,CFL08,LT 1850,Operativo,Operativo,0,2021-01-04 20:42:36


In [ ]:
df_excav_states['Name_eqmt'].unique().tolist()

['CFL08',
 'CRA01',
 'PAB05',
 'PAB06',
 'PAB08',
 'PAB10',
 'PAB11',
 'PAB12',
 'PHK09',
 'PHK13',
 'PHK14',
 'PHK15',
 'PRIM 2-1',
 'PRIM-1',
 'VEC01',
 'VIR01']

In [ ]:
# Esto va aca pq los chancadores tienen distinto nombre en states y dumps
dic_maestro['CHANC-1'] = 'PRIM-1'
dic_maestro['CHANC-2'] = 'PRIM 2-1'

In [ ]:
dic_maestro_reverse = {}
for key in dic_maestro.keys():
  dic_maestro_reverse[dic_maestro[key]] = key


In [ ]:
df_excav_states_list = df_excav_states.values.tolist()
new_column = []
for fila in df_excav_states_list:
  eqmt = fila[6]
  if eqmt in dic_maestro_reverse.keys():
    fila[6] = dic_maestro_reverse[eqmt]

columns = df_excav_states.columns
df_excav_states = pd.DataFrame(df_excav_states_list, columns=columns)

In [ ]:
df_excav_states[0:5]

,ShiftId,Year,Month,Day,Shift_type,FieldId,Name_eqmt,Eqmttype_eqmt,Description_status,estado,HH,dt_start
0,210103002,2021,01,03,2,00:49:52,PS08,LT 1850,Demora,Operativo,0,2021-01-03 20:49:52
1,210103002,2021,01,03,2,03:43:12,PS08,LT 1850,Reserva,No Operativo,3,2021-01-03 23:43:12
2,210104001,2021,01,04,1,02:01:57,PS08,LT 1850,Operativo,Operativo,2,2021-01-04 10:01:57
3,210104001,2021,01,04,1,11:34:31,PS08,LT 1850,Demora,Operativo,11,2021-01-04 19:34:31
4,210104002,2021,01,04,2,00:42:36,PS08,LT 1850,Operativo,Operativo,0,2021-01-04 20:42:36


In [ ]:
df_excav_states['Name_eqmt'].unique().tolist()

['PS08',
 'PExtCRA01',
 'PB05',
 'PB06',
 'PB08',
 'PB10',
 'PB11',
 'PB12',
 'PM09',
 'PM13',
 'PM14',
 'PM15',
 'CHANC-2',
 'CHANC-1',
 'PExtVEC01',
 'PExtVIR01']

In [ ]:
df_excav_states[df_excav_states['Name_eqmt']=='TExt01'][0:5]

,ShiftId,Year,Month,Day,Shift_type,FieldId,Name_eqmt,Eqmttype_eqmt,Description_status,estado,HH,dt_start


In [ ]:
dic_maestro

{'BOT_BANQUETA PALA 13': 'BANQUETA PALA 13',
 'BOT_BD2-3655E': 'BD2-3655E',
 'BOT_BD2-CURVA 1': 'BD2-CURVA 1',
 'BOT_BD2-CURVA 5': 'BD2-CURVA 5',
 'BOT_BD2-NIEVE': 'BD2-NIEVE',
 'BOT_BOT-3475 LIX': 'BOT-3475 LIX',
 'BOT_BOT-3505-LIX': 'BOT-3505-LIX',
 'BOT_BOT-3535-ESTERIL': 'BOT-3535-ESTERIL',
 'BOT_BOT-CURVALORO/ESTERIL': 'BOT-CURVALORO/ESTERIL',
 'BOT_BOT-EST-BCRATER': 'BOT-EST-BCRATER',
 'BOT_BOT-PLANTA ARIDOS ICV': 'BOT-PLANTA ARIDOS ICV',
 'BOT_BOT-PRISMA-INF': 'BOT-PRISMA-INF',
 'BOT_BOT-PRISMA-PLATAFORM.': 'BOT-PRISMA-PLATAFORM.',
 'BOT_BOT-RELLENO NODO VIAL': 'BOT-RELLENO NODO VIAL',
 'BOT_BOT-REM-3475-517': 'BOT-REM-3475-517',
 'BOT_BOT/NIVE-PRISMA1': 'BOT/NIVE-PRISMA1',
 'BOT_BOTADERO 3520': 'BOTADERO 3520',
 'BOT_BOTADERO PRISMA': 'BOTADERO PRISMA',
 'BOT_BOT_3475_BGAR': 'BOT_3475_BGAR',
 'BOT_BOT_3475_ENAEX': 'BOT_3475_ENAEX',
 'BOT_BOT_3475_ENAEX_C1': 'BOT_3475_ENAEX_C1',
 'BOT_BOT_3490_LIX': 'BOT_3490_LIX',
 'BOT_BOT_3535_LIX_RAMPA': 'BOT_3535_LIX_RAMPA',
 'BOT_BOT_3564_

# Descargar

In [ ]:
df_loads_simulador.to_csv(f'{rel_path}df_loads_simulador.csv')
df_maestro_quipo = pd.DataFrame(list_maestro, columns=['new_name', 'original','type'])
df_maestro_quipo.to_csv(f'{rel_path}maestro_equipo.csv')
df_excav_states.to_csv(f'{rel_path}df_excav_chanc_states.csv')

In [ ]:
df_loads_simulador['DumpLocation'].unique().tolist()

['CHANC-2',
 'BOT_BOT-CURVALORO/ESTERIL',
 'CHANC-1',
 'BOT_BOT_EST_3630',
 'BOT_BOT-3475 LIX',
 'BOT_BOT-PLANTA ARIDOS ICV',
 'BOT_BOT-3505-LIX',
 'BOT_BOT_REM_PISO_3490',
 'STOCK_STK_DONOSO_ML',
 'BOT_BD2-CURVA 1',
 'STOCK_STK_3600_ML',
 'STOCK_STK_DONOSO_AL',
 'BOT_RAMPA-3505-3520-LIX',
 'STOCK_STK_SFCO_AL',
 'BOT_BOT_EST_3645',
 'STOCK_STK_SFCO_ML',
 'BOT_BOT_EST_CRATER',
 'STOCK_STK_ENAEX_BOL',
 'STOCK_STK_ENAEX_AL',
 'BOT_RAMPA-3935',
 'BOT_BD2-CURVA 5',
 'STOCK_STK_BOLONES_3600ML',
 'BOT_BOT-RELLENO NODO VIAL',
 'BOT_BD2-NIEVE',
 'BOT_BOT_NIEVE_DON2',
 'STOCK_STK_3600_3620',
 'BOT_BOT_EST_TR13_3282',
 'BOT_BOT_EST_BOS_3282',
 'BOT_BOT_LIX_3475',
 'BOT_BOT-3535-ESTERIL',
 'BOT_BOT_EST_3535',
 'BOT_BOT_EST_INF6',
 'BOT_BOT_LIX_3505',
 'BOT_BOT_RELL_RAMPA_MASMIL',
 'BOT_BOT_EST_CURVA DEL LORO',
 'BOT_BOT_LIX_3505_PATA',
 'BOT_RAMPA-3460-3490-LIX',
 'STOCK_STK_PARRILLAS',
 'BOT_BOT_REM_RAMPA_3490',
 'BOT_RAMPA-3490',
 'BOT_BOT_REM_RAMPA_3475',
 'STOCK_STK_SFCO_AD',
 'BOT_BOT/NIVE-PR